# Code to analyze pbp data for fouls, assists etc

In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from pbpstats.resources.enhanced_pbp import Foul
from pbpstats.resources.enhanced_pbp import Turnover
from pbpstats.resources.enhanced_pbp import FieldGoal
from pbpstats.resources.enhanced_pbp import Rebound

fig_DIR = "../figs/pbp_related/"
source = "pbpstats"

In [ ]:
league = "NBA"
year = 2023
season_type = "Regular Season"
if league == "NBA":
    season = str(year)+'-'+str(year+1)[-2:]
else:
    season = str(year)
player_dict = get_players_pbp(league = league) # Creating Player Dictionary
team_dict = teams.get_teams() # Creating Team Dictionary
games_id = pbp_season(league=league,season_yr=str(year),season_type=season_type)
games_list = pbp_games(games_id)

# Frustration Fouls and Euro Fouls

## Estimate Euro Fouls

In [ ]:
t = 5
pos_store, TO_Miss_pID, Foul_pID,Foul_tID = [], [], [], []
# i, ii = 0, 0
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if isinstance(possession_event, Foul) and (isinstance(possession_event.previous_event, Turnover) or  (isinstance(possession_event.previous_event, FieldGoal) and not possession_event.previous_event.is_made)) and possession_event.seconds_since_previous_event <= t:
                pos_store.append(possession)
                TO_Miss_pID.append(possession_event.previous_event.player1_id)
                # print ("Turnover/Missed Shot Player: {0}".format(possession_event.previous_event.player1_id))
                Foul_pID.append(possession_event.player1_id)
                Foul_tID.append(possession_event.team_id)
                # print ("Foul Player: {0}".format(possession_event.player1_id))
                # ii += 1
                # print("ii = {0}".format(ii))
    # i += 1
    # print("i = {0}".format(i))
    # clear_output(wait=True)

In [ ]:
data_euro = rank_data_pbp(Foul_tID,player_dict,team_dict,sort="Team",var="Fouls")

## Estimate Frustration Fouls

In [ ]:
FFoul_pID =[]
for i in range(len(TO_Miss_pID)):
    if TO_Miss_pID[i] == Foul_pID[i]:
        FFoul_pID.append(Foul_pID[i])

In [ ]:
data_frust = rank_data_pbp(FFoul_pID,player_dict,team_dict,sort="Player",var="Fouls")

In [ ]:
title = "Frustration Fouls "+season
fig = plot_table_rank(data_frust,"Fouls",title=title,source="pbpstats")
fig.write_image(fig_DIR+f"{league}_frust_foul_{year}.png", scale=3)

In [ ]:
title = "Euro Fouls "+season
fig = plot_table_rank(data_euro,"Fouls",sort="Team",title=title,source="pbpstats")
fig.write_image(fig_DIR+f"{league}_euro_foul_{year}.png", scale=3)

# Coast to Coast Analysis

In basketball we say that whenever someone grabs a rebound, dribbles the ball up the court quickly and scores the basket, that the player is going coast to coast.
I came up with a method to estimate these events. I divide these events into two, coast-to-coast buckets i.e. when the player scores by themselves and coast-to-coast assists i.e when the player assists another player to score the ball.
For both types of events I have the same time cutoff of 5 seconds i.e. the player has to score/assist with 5 secs of grabbing the rebound.
Also for coast-to-coast buckets the FG has to bee within 5ft of the rim. NBA considers this distance as a rim attempt.

## Coast to Coast Assists

In [ ]:
t = 5
pos_store, Rebound_pID, Assist_pID = [],[],[]
i, ii = 0, 0
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if (isinstance(possession_event, FieldGoal) and possession_event.is_made and possession_event.is_assisted) and (isinstance(possession_event.previous_event, Rebound)) and possession_event.seconds_since_previous_event <= t:
                if possession_event.previous_event.player1_id == possession_event.player2_id:
                    if not possession_event.previous_event.oreb :
                        pos_store.append(possession)
                        Rebound_pID.append(possession_event.previous_event.player1_id)
                        Assist_pID.append(possession_event.player2_id)


In [ ]:
data_ca = rank_data_pbp(Assist_pID,player_dict,team_dict,sort="Player",var="AST")

## Coast to Coast Buckets

In [ ]:
t = 5
dist = 5
pos_store,Rebound_pID,FGM_pID = [],[],[]
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if (isinstance(possession_event, FieldGoal) and possession_event.is_made and not possession_event.is_assisted) and (isinstance(possession_event.previous_event, Rebound)) and possession_event.seconds_since_previous_event <= t:
                if not possession_event.previous_event.oreb and possession_event.distance<=dist :
                    if not possession_event.previous_event.oreb :
                        pos_store.append(possession)
                        Rebound_pID.append(possession_event.previous_event.player1_id)
                        FGM_pID.append(possession_event.player1_id)

In [ ]:
data_cb = rank_data_pbp(FGM_pID,player_dict,team_dict,sort="Player",var="FGM")

In [ ]:
title = "Coast to Coast Buckets " + season
footer = "Player scores <5ft within 5s of grabbing rebound"
fig = plot_table_rank(data_cb,"FGM",title=title,title_shift=0.02,title_font=14,footer=footer,source="pbpstats")
fig.write_image(fig_DIR+f"{league}_coast_buckets_{year}.png", scale=3)

In [ ]:
title = "Coast to Coast Assists " + season
footer = "Player assists within 5s of grabbing rebound"
fig = plot_table_rank(data_ca,"AST",title=title,footer=footer,source="pbpstats")
fig.write_image(fig_DIR+f"{league}_coast_assists_{year}.png", scale=3)

# Assists after Offensive rebounds

In [ ]:
ast_time = 14
pos_store, Rebound_pID, Assist_pID = [], [], []
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if (isinstance(possession_event, FieldGoal) and possession_event.is_made and possession_event.is_assisted) and (isinstance(possession_event.previous_event, Rebound)):
                if possession_event.previous_event.player1_id == possession_event.player2_id:
                    if possession_event.previous_event.oreb :
                        pos_store.append(possession)
                        Rebound_pID.append(possession_event.previous_event.player1_id)
                        Assist_pID.append(possession_event.player2_id)

In [ ]:
data_oa = rank_data_pbp(Assist_pID,player_dict,team_dict,sort="Player",var="AST")

In [ ]:
title = "Off Reb Assists " + season
footer = "Player assists after grabbing off rebound"
fig = plot_table_rank(data_oa,"AST",title=title,footer=footer,source="pbpstats")
fig.write_image(fig_DIR+f"{league}_oreb_assists_{year}.png", scale=3)